In [35]:
import sys
print(sys.version)
%load_ext autoreload
%autoreload 0


from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
from spread import get_spread_data

SALARY_FILE = './data/fanduel_9_20.csv'
WEEK = 3
ACTIVE_FILE = './data/active.csv'
print('ready')

3.7.11 (default, Jul  6 2021, 12:43:19) 
[Clang 12.0.5 (clang-1205.0.22.9)]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready


In [36]:
spread_df = get_spread_data(WEEK)
spread_df.info()

return cached data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 44 columns):
Unnamed: 0                      16 non-null int64
SeasonType                      16 non-null int64
Season                          16 non-null int64
Type                            16 non-null object
Week                            16 non-null int64
Timeframe                       16 non-null object
Day                             16 non-null object
DateTime                        16 non-null object
DayString                       16 non-null object
TimeString                      16 non-null object
DayOfWeek                       16 non-null object
DateTimeString                  16 non-null object
LastUpdated                     16 non-null object
HasStarted                      16 non-null bool
Status                          16 non-null object
Quarter                         0 non-null float64
AwayTeamID                      16 non-null int64
AwayTeam          

In [37]:
favor_map = {}
for index, row in spread_df.iterrows():

    favor_map[row['HomeTeam']] = row['PointSpread']
    favor_map[row['AwayTeam']] = -row['PointSpread']
    
if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']
    
print(favor_map)
print('Negative means favored') # https://madduxsports.com/how-to-read-nfl-odds-lines.html

{'HOU': 7.5, 'CAR': -7.5, 'BUF': -9.5, 'WAS': 9.5, 'CLE': -7.5, 'CHI': 7.5, 'DET': 7.5, 'BAL': -7.5, 'TEN': -5.0, 'IND': 5.0, 'KC': -7.0, 'LAC': 7.0, 'NE': -2.5, 'NO': 2.5, 'NYG': -3.5, 'ATL': 3.5, 'PIT': -4.5, 'CIN': 4.5, 'JAX': 7.0, 'ARI': -7.0, 'DEN': -10.5, 'NYJ': 10.5, 'LV': -5.0, 'MIA': 5.0, 'LAR': -1.5, 'TB': 1.5, 'MIN': 1.5, 'SEA': -1.5, 'SF': -4.0, 'GB': 4.0, 'DAL': -4.0, 'PHI': 4.0, 'JAC': 7.0}
Negative means favored


In [38]:

df = pd.read_csv(SALARY_FILE, na_values= '')
df = df.fillna(df.median())
print(df.describe())
df.to_csv(ACTIVE_FILE)
print('wrote ' + ACTIVE_FILE)
df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']
excluded_players = list(df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]['Name'])
# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {excluded_players}")
custom_adds = []#["Buffalo Bills"]# ["Rob Gronkowski"] #["Kyler Murray"] # [] #["Jake Kumerow"] 
if excluded_players:
    excluded_players.extend(custom_adds)


             FPPG      Played       Salary  Tier  Unnamed: 14  Unnamed: 15
count  839.000000  839.000000   839.000000   0.0          0.0          0.0
mean     4.071748    2.901073  4911.918951   NaN          NaN          NaN
std      5.398729    3.216305   969.794906   NaN          NaN          NaN
min     -0.750000    1.000000  3000.000000   NaN          NaN          NaN
25%      0.465000    1.000000  4500.000000   NaN          NaN          NaN
50%      2.258333    2.000000  4500.000000   NaN          NaN          NaN
75%      5.000000    2.000000  5300.000000   NaN          NaN          NaN
max     34.830002   18.000000  9700.000000   NaN          NaN          NaN
wrote ./data/active.csv
Excluding: ["D'Andre Swift", 'Josh Jacobs', 'Diontae Johnson', 'Carson Wentz', 'Baker Mayfield', 'Tua Tagovailoa', 'Odell Beckham Jr.', 'Jarvis Landry', 'Ryan Fitzpatrick', 'Andy Dalton', 'Darrell Henderson Jr.', 'Marcus Mariota', 'AJ McCarron', 'Chris Streveler', 'Nate Stanley', 'Sam Ehlinger', 'Kel

In [39]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
def get_nfl_positions():
    return [
        ['QB', 1, 1],
        ['RB', 2, 3],
        ['WR', 3, 4],
        ['TE', 1, 1],
        ['D', 1, 1]
    ]               

In [43]:
# Use salary data from csv as optimization basis.
players = salary_download.generate_players_from_csvs(
  salary_file_location=ACTIVE_FILE,
  game=rules.FAN_DUEL,
)

# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
block_function = lambda p: p.cost < 5000 or p.proj < 12 or favor_map[p.team] > -3

settings = OptimizerSettings(
    custom_rules=[
        CustomRule(
            group_a=lambda p: p,
            group_b=block_function, 
            comparison=lambda sum, a, b: sum(b) <= 1
        ),        
    ],
)

if excluded_players:
    constraints = LineupConstraints(banned=[excluded_players])
else:
    constraints = LineupConstraints()

ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000
ACTIVE_RULE_SET.defensive_positions = ['D']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.max_players_per_team = 9
ACTIVE_RULE_SET.roster_size = 9
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

roster = run(
    rule_set=ACTIVE_RULE_SET,
    player_pool=players,
    verbose=True,
    optimizer_settings=settings,
    constraints=constraints
)

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 1], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 0, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX'], 'defensive_positions': ['D'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None}
Optimal roster for: NFL
+----------+------------------+------+---------+--------+--------------------+----------+--------+
| Position | Player           | Team | Matchup | Salary |         Projection | vs. Avg. | Locked |
+----------+------------------+------+---------+--------+--------------------+----------+--------+
| QB       | Kyler Murray     | ARI  | ARI@JAC |  9,000 | 34.830001831054695 |     0.00 |        |
| RB       | Ty'Son Williams  | BAL  | BAL@DET |  6,400 | 16.899999618530273 |     0.00 |        |
| RB       | Zack Moss        | BUF  | WAS@BUF |  5,400 | 14.399999618530273

In [41]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:
#         return matches.iloc[0]['Id']
#     return None
roster.players[6].__dict__

AttributeError: 'NoneType' object has no attribute 'players'

In [ ]:
for r in roster.players:
    print(r.team, r.name, r.proj, r.average_score, r.cost,  r.kv_store['Injury Indicator'], r.kv_store['Injury Details'], r.kv_store['Played'])

In [ ]:
headers = []
players = []

for r in roster.players:
    headers.append(r.pos if r.pos != 'D' else 'DEF')
    p = f"{r.kv_store['Id']}:{r.name}"
    players.append(p)

with open('upload.csv', 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))



In [ ]:
output = pd.read_csv('upload.csv')
output

In [ ]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score

In [ ]:
expected_score = sum([m_score[h] for h in headers])
expected_score

In [ ]:
# Players that have non-null injury status.
df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]

In [ ]:
df[df['First Name'] == 'Jeff']

In [ ]:
# Potentially unaccounted positions
df[~df['Position'].isin(ALL_POSITIONS)]

In [ ]:
df[df['Position'] == 'D'][['FPPG', 'Nickname', 'Salary', 'Salary/FPPG']].sort_values('Salary/FPPG', ascending=False)

In [ ]:
df[df['Team'] == 'JAC']

In [ ]:
spread_df.iloc[1][['HomeTeam', 'PointSpread']]